In [1]:
import numpy as np
import sys

In [2]:
from misc import load, load_class_labels

dataset_root = "./dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train.csv', 1, sample_rate)
val_info = load(dataset_root + 'labels/validation.csv')
test_info = load(dataset_root + 'labels/test.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [3]:
from nn_filter import NN_filter

filter = NN_filter(train_info, train_dir, 'efficient_net', batch_size=25)

2023-12-27 08:43:30.787732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 08:43:30.787787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 08:43:30.806218: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 08:43:30.896175: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-27 08:43:31.885936: W tensorflow/compiler/tf2

In [4]:
features, labels, image_names = filter.fit_train()

100%|██████████| 3791/3791 [13:58<00:00,  4.52it/s] 


## Extract validation from trainig set

In [ ]:
import pandas as pd
train_info = load(dataset_root + 'train_info_dirty.csv', 1, 0.8)
val_info = load(dataset_root + 'train_info_dirty.csv', 1, 1)


same = val_info['filename'].isin(train_info['filename'])
val_info.drop(val_info[same].index, inplace=True)

#train_info.to_csv(dataset_root + '/labels/train.csv', header=False, index=False);
#val_info.to_csv(dataset_root + '/labels/validation.csv', header=False, index=False);

## Outlier detection using pca on NN features

In [5]:
features.shape

(94782, 48)

In [ ]:
pca = filter.fit_pca(features, components=9)

In [ ]:
significance_threshold = 0.8
np.set_printoptions(threshold=sys.maxsize)
cum_prod = filter.analyze_pca_components(pca)

#cum_prod = [pair for pair in cum_prod if pair[1] > significance_threshold][0]
#print('num of component that should be used: ', cum_prod[0])

In [ ]:
new_space = filter.pca_features(pca, features);

In [ ]:
from misc import group_stats
mean, cov = group_stats(new_space, labels)

In [ ]:
from image_discard import find_outliers_per_class
names, distances = find_outliers_per_class(mean, cov, new_space, labels, image_names)

In [ ]:
from image_discard import write_discarded_images
#write_discarded_images(names, class_labels, "./discarded_pca_nn_features/", train_dir)

In [8]:
len(train_info)

94782

## Outlier detection with NN features

In [12]:
from image_discard import write_discarded_images, find_outliers_iter, write_cleaned_csv
names, distances = find_outliers_iter(features, labels, image_names, threshold=11, num_iter=15)
write_discarded_images(names, class_labels, "./discarded_scaling_features_nn/", train_dir)

,filename,label
174,train_059928.jpg,0
163,train_059404.jpg,0
149,train_059720.jpg,0
310,train_059389.jpg,0
130,train_059947.jpg,0
...,...,...
94773,train_119918.jpg,250
94726,train_120057.jpg,250
94695,train_119881.jpg,250
94512,train_120171.jpg,250


In [13]:
clean_train_info = write_cleaned_csv(train_info, names, dataset_root)

In [ ]:
from misc import compute_independent_stats

clean_train_names = clean_train_info.iloc[:, 0].values
clean_train_labels = clean_train_info.iloc[:, 1].values

clean_features = features[np.isin(train_image_names, clean_train_names)]
clean_mean, clean_std = compute_independent_stats(clean_features, clean_train_labels)
np.savetxt(dataset_root + "stats/clean_train_mean.csv", clean_mean, delimiter=',')
np.savetxt(dataset_root + "stats/clean_train_std.csv", clean_std, delimiter=',')

In [47]:
filter = NN_filter(val_info, train_dir, 'efficient_net', batch_size=25)

In [48]:
val_features, val_labels_, val_names = filter.fit_train()

100%|██████████| 947/947 [03:22<00:00,  4.68it/s]


## Outlier detection with KNN

In [59]:
def center_scale_wrt_trainset(features, labels, train_mean, train_std):
    classes = np.unique(labels)
    scaled_features = np.copy(features)
    for c in classes:
        to_consider = labels == c
        scaled_features[to_consider] = (scaled_features[to_consider] - train_mean[c]) / train_std[c]
    return scaled_features

scaled_val = center_scale_wrt_trainset(val_features, val_labels_, clean_mean, clean_std)
val_m, val_s = compute_independent_stats(scaled_val, val_labels_)

array([[0.89467901, 1.09593021, 1.28283793, ..., 1.10490224, 2.82410312,
        1.70100398],
       [1.23450526, 1.60814618, 3.48508852, ..., 1.07607689, 1.82644926,
        0.82287366],
       [1.31997649, 1.12209728, 5.05572378, ..., 1.05889306, 1.22325106,
        0.93170014],
       ...,
       [2.18602816, 4.57953619, 2.5878339 , ..., 1.37459966, 0.6532724 ,
        1.46069825],
       [2.28202073, 3.53751669, 5.64597137, ..., 1.11097273, 3.14002712,
        0.97826855],
       [1.28576639, 1.19835186, 1.26416295, ..., 1.37390109, 1.33381668,
        0.81645432]])

In [60]:
names_v, distances_v = find_outliers_iter(scaled_val, val_labels_, val_names, threshold=11, num_iter=15)

In [63]:
sum = 0

for c in names_v:
    sum += len(c)
sum

3422

In [66]:
write_discarded_images(names_v, class_labels, "./discarded_validation_scaling_features_nn/", train_dir)
clean_val_info = write_cleaned_csv(val_info, names_v, dataset_root)

In [ ]:
filter = NN_filter(test_info, val_dir, 'efficient_net', batch_size=64)

In [ ]:
use_pca = False

In [ ]:
knn = filter.fit_knn(features, labels, use_pca);

In [ ]:
names = filter.filter_with_knn(knn, use_pca);
write_discarded_images(names, class_labels, "./discarded_knn/", train_dir)